<a href="https://colab.research.google.com/github/pauloigorms/web-scraping-olx/blob/main/web_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p><img alt="Ícone importando do portal FlatIcon" width="30" src="https://www.flaticon.com/svg/static/icons/svg/2742/2742225.svg" align="left" hspace="10px" vspace="0px"></p>

<h1><strong>Web Scraping | OLX E-commerce</strong></h1>

&emsp; Projeto independente sem fins lucrativos apenas com intuito de acrescentar e fixar conhecimentos em <b>web scraping</b>.


<br />

<i>Por Paulo Igor Moraes</i>

In [ ]:
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pandas as pd
import json

In [ ]:
def getDataOLX(pages = 2):

  params = {
    'authority': 'am.olx.com.br',
    'methoid': 'GET',
    'scheme': 'https',
    'user-agent': 'Mozilla/5.0'
  }

  for x in range(1, pages):

    page_result = Request(url_based + str(x), headers = params)
    page_parser = BeautifulSoup(urlopen(page_result).read(), 'html.parser')
    items = page_parser.find('div', {'class': 'sc-1fcmfeb-0'}).find_all('li')

    for item in items:

      content = item.find('a')

      if (content):
        try:
          
          price = content.find('div', {'class': 'fnmrjs-9'}).find('span').get_text()
          if (price):
            price = price.split('R$')[1]
            price = float(price.replace(".",""))
          else:
            price = 0

          link = content.get('href')
          
          data = {
            'id': content.get('data-lurker_list_id'),
            'link': link,
            'text': content.find('h2').get_text(),
            'image_home': content.find('img')['src'],
            'image_alt': content.find('img')['alt'],
            'price': price
          }
        
          if (link):
            item_page = Request(link, headers = params)
            item_parser = BeautifulSoup(urlopen(item_page).read(), 'html.parser')
            item_content = item_parser.find('div', {'class': 'duvuxf-0 h3us20-0 jAHFXn'})

            description = item_content.find('span', { 'class': 'sc-1sj3nln-1 eOSweo sc-ifAKCX cmFKIN' }).get_text()

            details = {}
            items_details = item_content.find('div', {'data-testid': 'ad-properties'}).find_all('div', {'class':'sc-hmzhuo'})
            for item in items_details:
              elements = item.find_all()
              if (elements[0].get_text() == 'Tamanho'):
                details[elements[0].get_text()] = elements[1].get_text().split("m")[0]
              else:
                details[elements[0].get_text()] = elements[1].get_text()
              json.dumps(details)

            data['details'] = details
            json.dumps(data)
            results.append(data)

          else:
            results.append(data)

        except:
          break
      else:
        break

In [ ]:
declaredVariable()
getDataOLX(pages = 2)
pd.json_normalize(results)

In [ ]:
def declaredVariable():
  url_based = 'https://am.olx.com.br/regiao-de-manaus/imoveis?o='
  results = []